# Lab 8 - BCC406

## REDES NEURAIS E APRENDIZAGEM EM PROFUNDIDADE

## Modelos Generativos

### Prof. Eduardo e Prof. Pedro

Objetivos:

- Predição de série temporal com redes recorrentes (RNN)

Data da entrega : 21/10

- Complete o código (marcado com ToDo) e quando requisitado, escreva textos diretamente nos notebooks. Onde tiver *None*, substitua pelo seu código.
- Execute todo notebook e salve tudo em um PDF **nomeado** como "NomeSobrenome-Lab6.pdf"
- Envie o PDF via google [FORM](https://forms.gle/zX7Va67EVzdXarbLA)

Este notebook é baseado em tensorflow e Keras.

# Predição de preço de criptomoedas com redes recorrentes

Informação sobre o Bitcoin : https://www.kaggle.com/ibadia/bitcoin-101-bitcoins-and-detailed-insights


O valor de uma criptomoeda, assim como um ativo funanceiro do mercado de ações, pode ser configurado com uma série temporal. Aqui, consideraremos o valor ponderado do preço diário do Bitcoin para constuir nossa série. O objetivo deste estudo é predizer o próximo valor, baseado nos últimos valores da criptomoeda. Para tal, usaremos de redes recorrentes, pois as mesmas tem memória, o que é importante quando se trata de dados sequenciais.


## Carregando os pacotes

In [1]:
# Importa as bibliotecas necessárias
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import seaborn as sns
py.init_notebook_mode(connected=True)
%matplotlib inline

Vamos usar o pacote ***quandl*** para baixar diretamente dados fornecidos por uma corretora de criptomoedas (Kraken).

In [2]:
!pip install quandl


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Carregando os dados

'KsfygqkVu3y9sL3c1rB'

In [8]:


# baixa os dados da exchange Kraken, até o período atual.
import quandl
#quandl.api_config.read_key("APIkey.txt")
API_KEY=open("APIkey.txt").read()
data = quandl.get_table('BCHARTS/KRAKENUSD', returns='pandas',  authtoken=API_KEY)
#data = quandl.get('NSE/OIL')


LimitExceededError: ignored

## Entendendo os dados

In [ ]:
#exibe as primeiras linahs 
data.head()

In [ ]:
data.info()

In [ ]:
# verifica os últimos dados. Repare na data. Deve ter dados atuais (Jun / 2021).
data.tail()

Repare que temos dados de abertura do pregão, fechamento, valor mais alto, valor mais baixo, volume diário do bitcoin e de todas as criptomoedas combinadas. E também, temos os preço ponderado pelos valores de compra/venda de um período, que em nosso caso é diário. Para facilitar, vamos usar o valor ponderado.

## Plotando os dados

In [ ]:
# imprima os dados
pyplot.plot(data['Weighted Price'])

## Pré-processamento dos dados

In [ ]:
#existem alguns pontos com valor zero (outliers), vamos trocar por NaN e depois chamar um método para preencher os valores vazios
data['Weighted Price'].replace(0, np.nan, inplace=True)
data['Weighted Price'].fillna(method='ffill', inplace=True)

In [ ]:
# imprima novamente e observe que não existe mais estes outliers.
pyplot.plot(data['Weighted Price'])

In [ ]:
# vamos usar o preço ponderado como entrada para nossa rede recorrente
# como já vimos, eh sempre bom normalizar os dados para ajudar na convergência do treinamento
# Normaliza na faixa entre [0 e 1]
from sklearn.preprocessing import MinMaxScaler
values = data['Weighted Price'].values.reshape(-1,1)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [ ]:
# vamos deixar 70% para treino e 30% para teste. Observe que temos mais de 6 anos de dados.
train_size = int(len(scaled) * 0.7)
test_size = len(scaled) - train_size
train, test = scaled[0:train_size,:], scaled[train_size:len(scaled),:]
print(len(train), len(test))

Vamos considerar uma janela de um único dia para efetuar a predição. Para isso, use a função create_dataset(..) e deixe o parâmetro look_back=1. O parâmetro look_back controla a quantidade de dados que vai fazer parte da janela de entrada para a rede. Estude e entenda o que a função faz.

In [ ]:
#função para criar os conjuntos de dados de treino
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

In [ ]:
# entra com janela de 1 único valor
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
trainX.shape

In [ ]:
# reshape para formato de entrada da rede neural (instancias, 1, 1)
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
trainX.shape

## Projeto de uma rede recorrente

Projete uma rede recorrente, usando alguma das camadas abaixo:

```
  tf.keras.layers.LSTM
  tf.keras.layers.GRU
  tf.keras.layers.RNN
```

As camadas recorrentes (LSTM, GRU, RNN) podem ser bidirecionais ou simpels, por exemplo, uma camada LSTM com 32 unidades e bidirecional:

 ```
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))
 ```

Você também pode usar dropout e camadas densas em seu modelo. 

Experimente três arquiteturas (rasas e profundas) e pelo menos dois algoritmos de otimização. Documente os resultados em uma tabela e anexe.

 Por exemplo, você pode usar um modelo raso como o abaixo:

```
 np.random.seed(42)
 tf.random.set_seed(42)


 model_1 = Sequential([
   LSTM(128,input_shape=[None,1]),
   Dense(1)
 ])
```

Com uma função de custo **Mean Square Error** e o algoritmo de otimização **ADAM**:

```
 model_1.compile(loss='mse',optimizer = 'adam')
```

Ou pode usar um modelo profundo, mais complexo como o abaixo:

```
 model = Sequential()
 model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
 model.add(Dense(units = 64, activation='relu'))
 model.add(Dropout(dropout_rate))
 model.add(Dense(units = 1))
```

O **erro médio quadrático** deste último modelo, com o otimizador **ADAM** e **erro médio quadrático** como função de custo deve resultar em:

```
 Test Root Mean Square Error (RMSE): 380.139
```

### Observações
1. **Seu RMSE pode ser diferente devido aos dados usados.**
2. **Use modelos diferentes dos de exemplo!**


 

## ToDo: Projetando os seus modelos (30pt)

#### Modelo 1:

In [ ]:

# ToDO : projete o modelo aqui


#### Modelo 2:

In [ ]:

# ToDO : projete o modelo aqui


#### Modelo 3:

In [ ]:

# ToDO : projete o modelo aqui


## ToDo: Função de custo (10pt)

Como é um problema de regressão, usaremos funções de custo apropriadas. Você pode usar, por exemplo, *Mean Absolute Error* (mae) ou *Mean Squared Error* (mse). 

**ToDo:** Estude as funções de custo MAE e MSE. Qual das duas funções você usaria. Justifique sua escolha. Repare que vamos avaliar os modelos pela métrica *Root Mean Square Error* (RMSE).

## ToDo: Função para treinar o seu modelo (15pt)

In [ ]:
# Função para treinar o modelo
def train_model(model, loss, optimizer, trainX, trainY):
  # Compile o modelo : atenção para a função de CUSTO. Abaixo um exemplo de uso da 'mae'
  model.compile(loss=loss, optimizer=optimizer)

  #treine o modelo
  history = model.fit( # todo...

  # plote a curva de custo
  pyplot.plot(history.history['loss'], label='train')
  pyplot.plot(history.history['val_loss'], label='test')
  pyplot.legend()

  pyplot.show()



## Função para avaliar o seu modelo


In [ ]:
# Avaliando o modelo treinado
def evaluate_model(model, testX, testY):

  # plote as curvas, valor real e valor predito no mesmo gráfico
  yhat = model.predict(testX)
  pyplot.title('Curva do valor real e valor predito na escala usado no treino')
  pyplot.plot(yhat, label='predict')
  pyplot.plot(testY, label='true')
  pyplot.legend()
  pyplot.show()
  
  # os valores foream normalizados para o treinamento. 
  # Veja que para fazer sentido, eles devem voltar para a escala original.
  # Volta para escala em US dollar :
  yhat_inverse = scaler.inverse_transform(yhat.reshape(-1, 1))
  testY_inverse = scaler.inverse_transform(testY.reshape(-1, 1))

  # calcula o RMSE 
  rmse = sqrt(mean_squared_error(testY_inverse, yhat_inverse))
  print('Test RMSE: %.3f' % rmse)

  # valor em US dollar
  pyplot.title('Curva do valor real e valor predito em US dollar')
  pyplot.plot(yhat_inverse, label='predict')
  pyplot.plot(testY_inverse, label='actual', alpha=0.5)
  pyplot.legend()
  pyplot.show()

## ToDo: Treinando e avaliando o seu modelo (15pt)

### Modelo 1

In [ ]:

# Modelo 1


### Modelo 2

In [ ]:

# Modelo 2


### Modelo 3

In [ ]:

# Modelo 3


## Prevendo o próximo dia

In [ ]:
# https://www.tensorflow.org/tutorials/structured_data/time_series
def create_time_steps(length):
  time_steps = []
  for i in range(-length, 0, 1):
    time_steps.append(i)
  return time_steps

def baseline(history):
  return np.mean(history)
  
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  pyplot.title(title)
  for i, x in enumerate(plot_data):
    if i:
      pyplot.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      pyplot.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  pyplot.legend()
  pyplot.xlim([time_steps[0], (future+5)*2])
  pyplot.xlabel('Time-Step')
  return pyplot

### ToDo: Função para predizer o próximo dia (15pt)

In [ ]:
def predict_next_day(model, testX, testY):
  # os valores foream normalizados para o treinamento. 
  # Veja que para fazer sentido, eles devem voltar para a escala original.
  # Volta para escala em US dollar :
  yhat_inverse = scaler.inverse_transform(model_1.predict(testX).reshape(-1, 1))
  testY_inverse = scaler.inverse_transform(testY.reshape(-1, 1))

  # na base de teste, plote até a instância 200 e tente predizer a instância futura: 201
  # use a função show_plot
  show_plot([testY_inverse[0:200], yhat_inverse[201],
             baseline(testY_inverse[201])], 1, 'Predição do dia seguinte')
  
  print('valor predito do dia 201: ', yhat_inverse[201])
  print('Valor real do dia 201: ', testY_inverse[201])

#### Modelo 1:

In [ ]:

# ToDo: seu código


#### Modelo 2:

In [ ]:

# ToDo: seu código


#### Modelo 3:

In [ ]:

# ToDo: seu código


## ToDo: Resultados (15pt)

Coloque os valores dos modelos em uma tabela. Em cada coluna, informe qual a função de custo utilizada, qual otimizador e o erro na partição de teste em RMSE.

```
   ------------------------------------------------------
   | Modelo   | Função de Custo | Otimizador | RMSE     |
   ------------------------------------------------------
   | Modelo 1 |                 |            |          |
   | Modelo 2 |                 |            |          |
   | Modelo 3 |                 |            |          |
   ------------------------------------------------------
```
